In [2]:
! pip install pyspark

  Using cached pyspark-3.5.0.tar.gz (316.9 MB)
  Preparing metadata (setup.py) ... done
  Using cached py4j-0.10.9.7-py2.py3-none-any.whl (200 kB)
  DEPRECATION: pyspark is being installed using the legacy 'setup.py install' method, because it does not have a 'pyproject.toml' and the 'wheel' package is not installed. pip 23.1 will enforce this behaviour change. A possible replacement is to enable the '--use-pep517' option. Discussion can be found at https://github.com/pypa/pip/issues/8559
  Running setup.py install for pyspark ... done


In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, count, mean, stddev, min, max

# Create a Spark session
spark = SparkSession.builder.appName("PySparkProfile").getOrCreate()

# Load your dataset into a PySpark DataFrame
# Replace 'your_dataset.csv' with your actual dataset file
df_spark = spark.read.csv('newdata.csv', header=True, inferSchema=True)

# Display basic statistics from the PySpark DataFrame, excluding Date_of_joining
df_stats = df_spark.select(
    *[count(col(c)).alias(c + "_count") for c in df_spark.columns],
    *[mean(col(c)).alias(c + "_mean") for c in df_spark.columns if c != "Date_of_joining"],
    *[stddev(col(c)).alias(c + "_stddev") for c in df_spark.columns if c != "Date_of_joining"],
    *[min(col(c)).alias(c + "_min") for c in df_spark.columns if c != "Date_of_joining"],
    *[max(col(c)).alias(c + "_max") for c in df_spark.columns if c != "Date_of_joining"]
)

# Show the statistics
df_stats.show(truncate=False)


+----------+---------+------------+---------------------+---------+--------+-----------+-----------+------------------+------------------+--------+-------+----------+--------+-------+----------+
|Name_count|Age_count|Salary_count|Date_of_joining_count|Name_mean|Age_mean|Salary_mean|Name_stddev|Age_stddev        |Salary_stddev     |Name_min|Age_min|Salary_min|Name_max|Age_max|Salary_max|
+----------+---------+------------+---------------------+---------+--------+-----------+-----------+------------------+------------------+--------+-------+----------+--------+-------+----------+
|10        |10       |10          |10                   |NULL     |37.9    |43316.6    |NULL       |11.454741861391339|11282.762469064628|Craig   |23     |24560.0   |smith   |56     |61000.0   |
+----------+---------+------------+---------------------+---------+--------+-----------+-----------+------------------+------------------+--------+-------+----------+--------+-------+----------+



In [1]:
from pyspark.sql import SparkSession
from pandas_profiling import ProfileReport

# Create a Spark session
spark = SparkSession.builder.appName("PySparkProfile").getOrCreate()

# Load dataset into a PySpark DataFrame
df_spark = spark.read.csv('newdata.csv', header=True, inferSchema=True)

# Generate a profile report
profile = ProfileReport(df_stats.toPandas())

# Save the profile report as an interactive HTML file
profile.to_file("pyspark_report.html")


/tmp/ipykernel_16543/3630016228.py:2: DeprecationWarning: `import pandas_profiling` is going to be deprecated by April 1st. Please use `import ydata_profiling` instead.
  from pandas_profiling import ProfileReport
24/02/12 14:27:55 WARN Utils: Your hostname, sigmoid-20JVA03PIG resolves to a loopback address: 127.0.1.1; using 172.29.170.199 instead (on interface wlp5s0)
24/02/12 14:27:55 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/02/12 14:27:56 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


NameError: name 'df_stats' is not defined

In [8]:
! pip install spark_df_profiling


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.8/91.8 kB 896.0 kB/s eta 0:00:00 0:00:01


In [17]:
# Without using pandas_profiling

from pyspark.sql import SparkSession
from pyspark.sql.functions import mean, stddev, min, max, count, when, col



spark = SparkSession.builder.appName("PySparkProfile").getOrCreate()

df_spark = spark.read.csv('newdata.csv', header=True, inferSchema=True)

metrics = {
    'row_count': df_spark.count(),
    'property_count': len(df_spark.columns),
    'empty_properties': {c: df_spark.where(col(c).isNull()).count() for c in df_spark.columns},
    'average': {c: df_spark.select(mean(col(c))).first()[0] for c in df_spark.columns if c != "Date_of_joining"},
    'sum': {c: df_spark.agg({c: "sum"}).first()[0] for c in df_spark.columns if c != "Date_of_joining"},
    'standard_deviation': {c: df_spark.select(stddev(col(c))).first()[0] for c in df_spark.columns if c != "Date_of_joining"},
    'min': {c: df_spark.select(min(col(c))).first()[0] for c in df_spark.columns if c != "Date_of_joining"},
    'max': {c: df_spark.select(max(col(c))).first()[0] for c in df_spark.columns if c != "Date_of_joining"},
    'sample_values': {c: df_spark.select(col(c)).limit(5).rdd.flatMap(lambda x: x).collect() for c in df_spark.columns},

}

# Generate HTML report
html_report = "<html><head><title>PySpark Data Profile Report</title></head><body>"

# Add metrics to HTML report
html_report += "<h2>Data Profiling Metrics:</h2>"
html_report += "<table border='1'><tr><th>Metric</th><th>Value</th></tr>"
for metric, value in metrics.items():
    if isinstance(value, dict):
        value_str = ', '.join(f"{k}: {v}" for k, v in value.items())
    else:
        value_str = str(value)
    html_report += f"<tr><td>{metric}</td><td>{value_str}</td></tr>"
html_report += "</table>"

# End HTML report
html_report += "</body></html>"

# Save the profile report as an interactive HTML file
with open("profile_report899.html", "w") as f:
    f.write(html_report)

# print("Data Profiling Metrics:")
# for metric, value in metrics.items():
#     print(f"{metric}: {value}")


Data Profiling Metrics:
row_count: 10
property_count: 4
empty_properties: {'Name': 0, 'Age': 0, 'Salary': 0, 'Date_of_joining': 0}
average: {'Name': None, 'Age': 37.9, 'Salary': 43316.6}
sum: {'Name': None, 'Age': 379, 'Salary': 433166.0}
standard_deviation: {'Name': None, 'Age': 11.454741861391339, 'Salary': 11282.762469064628}
min: {'Name': 'Craig', 'Age': 23, 'Salary': 24560.0}
max: {'Name': 'smith', 'Age': 56, 'Salary': 61000.0}
sample_values: {'Name': ['booker', 'grey', 'johnson', 'jenkins', 'smith'], 'Age': [23, 56, 43, 34, 32], 'Salary': [55000.0, 45000.0, 45400.0, 32400.0, 30000.0], 'Date_of_joining': [datetime.date(2022, 2, 1), datetime.date(2023, 7, 7), datetime.date(2022, 3, 11), datetime.date(2019, 8, 16), datetime.date(2020, 10, 4)]}
